In [2]:
!pip install diffusers transformers accelerate safetensors insightface opencv-python onnxruntime

import torch
from diffusers import StableDiffusionControlNetInpaintPipeline, ControlNetModel
from diffusers.utils import load_image
import cv2
import insightface

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.6 MB/s eta 0:00:00


In [3]:
# Load ControlNet for pose/depth (helps modify body too)
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-depth", torch_dtype=torch.float16
)

# Load SDXL inpainting pipeline
pipe = StableDiffusionControlNetInpaintPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    controlnet=controlnet,
    torch_dtype=torch.float16,
).to("cuda")

pipe.enable_xformers_memory_efficient_attention()

# --------------------
# Load image & create mask (mask = person only)
# --------------------
input_image = load_image("your_image.jpg")

# Use insightface for face+body segmentation
app = insightface.app.FaceAnalysis(name="buffalo_l")
app.prepare(ctx_id=0, det_size=(640,640))
faces = app.get(cv2.cvtColor(cv2.imread("your_image.jpg"), cv2.COLOR_BGR2RGB))

mask = cv2.cvtColor(cv2.imread("your_image.jpg"), cv2.COLOR_BGR2GRAY)
mask[:] = 0
for f in faces:
    x,y,w,h = f.bbox.astype(int)
    cv2.rectangle(mask, (x,y), (x+w,y+h+200), 255, -1)  # include torso

cv2.imwrite("mask.png", mask)




config.json:   0%|          | 0.00/920 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.45G [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

text_encoder_2/model.safetensors:   0%|          | 0.00/2.78G [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

vae_1_0/diffusion_pytorch_model.safetens(…):   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
You have disabled the safety checker for <class 'diffusers.pipelines.controlnet.pipeline_controlnet_inpaint.StableDiffusionControlNetInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
# --------------------
# Run age transformation
# --------------------
prompt = "a 65 year old version of the same person, realistic skin, wrinkles, consistent body aging"
negative_prompt = "distorted face, deformed body, identity change"

result = pipe(
    prompt=prompt,
    image=input_image,
    mask_image=load_image("mask.png"),
    negative_prompt=negative_prompt,
    num_inference_steps=40,
    guidance_scale=7.5,
).images[0]

result.save("aged_person.jpg")